In [1]:
import numpy as np
from patsy import dmatrices
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter

%config Completer.use_jedi = False 
%matplotlib widget

In [2]:
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos.pick"
time_plate_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\global_hypha_infos.pick"
global_hypha_info = pd.read_pickle(path_save)
path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_hypha_infos.pick"
time_hypha_info = pd.read_pickle(path_save)


In [3]:
plate = 152
select = time_plate_info.loc[(time_plate_info['Plate']==plate)&(time_plate_info['out_study']==0)]

In [4]:
fig, ax = plt.subplots()

select.plot('time_since_begin','L_rh',ax = ax)
select.plot('time_since_begin','L_BAS',ax = ax)
select.plot('time_since_begin','tot_length_study',ax = ax)
ax.plot(select['time_since_begin'],select['L_rh']+select['L_BAS'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
fig, ax = plt.subplots()
# ax.plot(select['time_since_begin'],(select['L_rh']+select['L_BAS'])/select['tot_length_study'])
ax.plot(select['time_since_begin'],(select['L_BAS'])/select['tot_length_study'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig, ax = plt.subplots()
plates = [76,94,102,152,792,799,800,808,80]

for plate in plates:
    select = time_plate_info.loc[(time_plate_info['Plate']==plate)&(time_plate_info['tot_length_study']>1000)&(time_plate_info['L_BAS']>0)]
    ax.plot(select['time_since_begin'],(select['L_BAS'])/(select['tot_length_study']),label = plate,alpha = 0.8)
ax.set_ylim((0,0.6))
ax.set_xlabel('time since begin (h)')
ax.set_ylabel('BAS length to total length ratio')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plates = [94,80,10,76,102,26,152,792]
plates = [94,76,102,152,792]
t0s = {94 : 22,102 : 32,152:27,792 : 44,76:13}
t1s = {94 : 46,102 : 60,152:51,792 : 61,76:28}
fig, ax = plt.subplots()
ax2 = ax.twinx()
# ax.plot(select['time_since_begin'],(select['L_rh']+select['L_BAS'])/select['tot_length_study'])
for plate in plates:
    select = time_plate_info.loc[(time_plate_info['Plate']==plate)&(time_plate_info['out_study']==0)]
    to_plot = time_plate_info.loc[(time_plate_info['Plate']==plate)].copy()
    to_plot = to_plot.loc[to_plot['out_study']==0]
    to_plot['log_length_study'] = np.log2(to_plot['tot_length_study'].astype(float))

    to_plot['local_growth_rate'] = np.gradient(to_plot['log_length_study'],to_plot['time_since_begin'])*24
    to_plot = to_plot.set_index('t')
    table_hypha = global_hypha_info.loc[global_hypha_info['Plate']==plate]

    series = table_hypha.loc[table_hypha['timestep_anastomosis'].isnull()==False].groupby('timestep_anastomosis')['timestep_anastomosis'].count()
    to_plot['num_anastomosis'] = series
    to_plot['num_anastomosis'] = to_plot['num_anastomosis'].fillna(0)
    to_plot['cum_num_anastomosis'] = np.cumsum(to_plot['num_anastomosis'])

#     ax2.scatter(select['time_since_begin']-t1s[plate],(select['L_BAS'])/(select['tot_length_study']+1),label = plate)
    ax2.scatter(to_plot['time_since_begin']-t1s[plate],(to_plot['L_BAS'])/(to_plot['tot_length_study']),label = plate)

    y = 'tot_length_study'
    f = interp1d(to_plot['time_since_begin'], to_plot[y])
    deb = np.min(to_plot['time_since_begin'])
    end = np.max(to_plot['time_since_begin'])
    frequ = 1
    av_lapse = 311
    x = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x,'total_length_m':f(x)})
    resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24
    y = 'tot_length_study'
    f = interp1d(to_plot['time_since_begin'], (to_plot['L_BAS'])/(to_plot['tot_length_study']))
    av_lapse = 311

    resampled2 = pd.DataFrame({'time':x,'bas_ratio':f(x)})
    resampled2['bas_ratio_smooth']= savgol_filter(resampled2['bas_ratio'], av_lapse, 2)
    ax.plot(resampled['time']-t1s[plate],resampled['local_growth_rate'])
    ax2.plot(resampled['time']-t1s[plate],resampled2['bas_ratio_smooth'],linestyle='dotted')


    ax.set_ylim((0,4))
plt.legend()
ax.set_xlabel('time (h)')
ax.set_ylabel('relative growth rate $(.day-1)$')
ax2.set_ylabel('BAS length to total length ratio')
# ax2.set_ylabel('total number of anastomosis / unit length (.um-1)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'BAS length to total length ratio')

In [6]:
plates = [94,80,10,76,102,26,152,792]
plates = [94,76,102,152,792]
t0s = {94 : 22,102 : 32,152:27,792 : 44,76:13}
t1s = {94 : 46,102 : 60,152:51,792 : 61,76:28}
fig, ax = plt.subplots()
ax2 = ax.twinx()
# ax.plot(select['time_since_begin'],(select['L_rh']+select['L_BAS'])/select['tot_length_study'])
for plate in plates:
    select = time_plate_info.loc[(time_plate_info['Plate']==plate)&(time_plate_info['out_study']==0)]
    to_plot = time_plate_info.loc[(time_plate_info['Plate']==plate)].copy()
    to_plot = to_plot.loc[to_plot['out_study']==0]
    to_plot['log_length_study'] = np.log2(to_plot['tot_length_study'].astype(float))

    to_plot['local_growth_rate'] = np.gradient(to_plot['log_length_study'],to_plot['time_since_begin'])*24
    to_plot = to_plot.set_index('t')
    table_hypha = global_hypha_info.loc[global_hypha_info['Plate']==plate]

    series = table_hypha.loc[table_hypha['timestep_anastomosis'].isnull()==False].groupby('timestep_anastomosis')['timestep_anastomosis'].count()
    to_plot['num_anastomosis'] = series
    to_plot['num_anastomosis'] = to_plot['num_anastomosis'].fillna(0)
    to_plot['cum_num_anastomosis'] = np.cumsum(to_plot['num_anastomosis'])

#     ax2.scatter(select['time_since_begin']-t1s[plate],(select['L_BAS'])/(select['tot_length_study']+1),label = plate)
    ax2.scatter(to_plot['time_since_begin']-t1s[plate],(to_plot['cum_num_anastomosis'])/(to_plot['tot_length_study']),label = plate)

    y = 'tot_length_study'
    f = interp1d(to_plot['time_since_begin'], to_plot[y])
    deb = np.min(to_plot['time_since_begin'])
    end = np.max(to_plot['time_since_begin'])
    frequ = 1
    av_lapse = 311
    x = np.linspace(deb,end,1000)
    resampled = pd.DataFrame({'time':x,'total_length_m':f(x)})
    resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
    resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
    resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24

    ax.plot(resampled['time']-t1s[plate],resampled['local_growth_rate'])

    ax.set_ylim((0,4))
plt.legend()
ax.set_xlabel('time (h)')
ax.set_ylabel('relative growth rate $(.day-1)$')
# ax2.set_ylabel('length ratio BAS/total')
ax2.set_ylabel('total number of anastomosis / unit length (.um-1)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'total number of anastomosis / unit length (.um-1)')

In [19]:
to_plot = time_plate_info.loc[(time_plate_info['Plate']==plate)].copy()
to_plot = to_plot.loc[to_plot['out_study']==0]
to_plot['log_length_study'] = np.log2(to_plot['tot_length_study'].astype(float))

to_plot['local_growth_rate'] = np.gradient(to_plot['log_length_study'],to_plot['time_since_begin'])*24

y = 'tot_length_study'
f = interp1d(to_plot['time_since_begin'], to_plot[y])
deb = np.min(to_plot['time_since_begin'])
end = np.max(to_plot['time_since_begin'])
frequ = 1
av_lapse = 311
x = np.linspace(deb,end,1000)
resampled = pd.DataFrame({'time':x,'total_length_m':f(x)})
reg = LinearRegression().fit(x.reshape((-1, 1)), f(x))
resampled['total_length_m_smooth']= savgol_filter(resampled['total_length_m'], av_lapse, 3)
resampled['log_total_length_m_smooth']=np.log2(resampled['total_length_m_smooth'])
resampled['local_growth_rate'] = resampled['log_total_length_m_smooth'].diff(10)/resampled['time'].diff(10)*24

fig, ax = plt.subplots()
ax.plot(resampled['time'],resampled['local_growth_rate'])
ax.plot(to_plot['time_since_begin'],to_plot['local_growth_rate'])

ax.set_ylim((0,2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 2.0)

In [29]:
table_hypha = global_hypha_info.loc[global_hypha_info['Plate']==94]
series = table_hypha.loc[table_hypha['timestep_anastomosis'].isnull()==False].groupby('timestep_anastomosis')['av_width_final'].count()
fig, ax = plt.subplots()
ax.plot(series[:-2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
series = table_hypha.loc[table_hypha['timestep_anastomosis'].isnull()==False].groupby('timestep_anastomosis')['timestep_anastomosis'].count()


In [33]:
series

timestep_anastomosis
0      5
1      2
3      2
4      2
8      1
      ..
94    18
95    14
96    18
97    16
98    22
Name: timestep_anastomosis, Length: 91, dtype: int64